# [BlenderBot 2.0: An open source chatbot that builds long-term memory and searches the internet](https://parl.ai/projects/blenderbot2/)
* [Wizard Of Internet models](https://parl.ai/docs/zoo.html#wizard-of-internet-models)
* [Unable to use BlenderBot2 model #3808](https://github.com/facebookresearch/ParlAI/issues/3808)
* [ParlAI_SearchEngine](https://github.com/JulesGM/ParlAI_SearchEngine)

In [ ]:
!git clone https://github.com/RodolfoFigueroa/ParlAI.git ./ParlAI
!git clone https://github.com/pytorch/fairseq ./fairseq

Cloning into './ParlAI'...
remote: Enumerating objects: 40208, done.
remote: Counting objects: 100% (1484/1484), done.
remote: Compressing objects: 100% (746/746), done.
remote: Total 40208 (delta 857), reused 1266 (delta 723), pack-reused 38724
Receiving objects: 100% (40208/40208), 87.71 MiB | 24.80 MiB/s, done.
Resolving deltas: 100% (28575/28575), done.
Cloning into './fairseq'...
remote: Enumerating objects: 28651, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 28651 (delta 54), reused 94 (delta 46), pack-reused 28533
Receiving objects: 100% (28651/28651), 12.05 MiB | 27.97 MiB/s, done.
Resolving deltas: 100% (21503/21503), done.


In [ ]:
!pip install -r ./ParlAI/requirements.txt
%cd ./ParlAI/
!python ./setup.py develop
%cd ../fairseq
!pip install --editable ./
%cd ..

     |████████████████████████████████| 131 kB 8.3 MB/s 
     |████████████████████████████████| 7.6 MB 45.2 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 262 kB 75.8 MB/s 
     |████████████████████████████████| 547 kB 63.4 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 70 kB 10.2 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 453 kB 74.3 MB/s 
     |████████████████████████████████| 123 kB 79.6 MB/s 
     |████████████████████████████████| 784 kB 56.6 MB/s 
     |████████████████████████████████| 294 kB 76.9 MB/s 
     |████████████████████████████████| 1.5 MB 31.6 MB/s 
     |████████████████████████████████| 20.0 MB 1.2 MB/s 
     |████████████████████████████████| 10.5 MB 61.5 MB/s 
     |████████████████████████████████| 234 kB 67.0 MB/s 
     |████████████████████████████████| 58 kB 7.8 MB/s 
     |█████████████████

/content/ParlAI
running develop
running egg_info
creating parlai.egg-info
writing parlai.egg-info/PKG-INFO
writing dependency_links to parlai.egg-info/dependency_links.txt
writing entry points to parlai.egg-info/entry_points.txt
writing requirements to parlai.egg-info/requires.txt
writing top-level names to parlai.egg-info/top_level.txt
writing manifest file 'parlai.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'parlai.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/parlai.egg-link (link to .)
Adding parlai 1.3.0 to easy-install.pth file
Installing parlai script to /usr/local/bin

Installed /content/ParlAI
Processing dependencies for parlai==1.3.0
Searching for huggingface-hub==0.0.12
Best match: huggingface-hub 0.0.12
Adding huggingface-hub 0.0.12 to easy-install.pth file
Installing huggingface-cli script to /usr/local/bin

Using /usr/local/lib/python3.7/dist-packages
Searching for jsonlines==1.2.0
Best match: jsonlin

In [ ]:
!git clone https://github.com/JulesGM/ParlAI_SearchEngine.git
%cd ParlAI_SearchEngine
!pip install bs4 fire html2text google-search parlai rich requests

Cloning into 'ParlAI_SearchEngine'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 37 (delta 11), reused 33 (delta 7), pack-reused 0
Unpacking objects: 100% (37/37), done.
/content/ParlAI_SearchEngine
     |████████████████████████████████| 87 kB 4.7 MB/s 
     |████████████████████████████████| 208 kB 51.0 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=0a6f5bb17826cb84ee71de8b7fa5cc38dcaca8c328f3d903b800e3215879b97b
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
%%writefile search_server.py
"""
A web search server for ParlAI, including Blenderbot2.

- Uses `html2text` to strip the markup out of the page.
- Uses `beautifulsoup4` to parse the title.
- Currently only uses the `googlesearch` module to query Google for urls, but is coded
in a modular / search engine agnostic way to allow very easily add new search engine support.

Using the `googlesearch` module is very slow because it parses webpages instead of querying webservices. This is fine for playing with the model, but makes that searcher unusable for training or large scale inference purposes.

To be able to train, one would just have to for example pay for Google Cloud or Microsoft Azure's search services, and derive the Search class to query them.

### Quick Start:

```bash
python search_server.py serve --host 0.0.0.0:8080
```

You can then for example start Blenderbot2 passing the server's address and ip as arguments:
```
parlai --model-file zoo:blenderbot2/blenderbot2_3B/model --search_server 0.0.0.0:8080
```


### Testing the server:
You need to already be running a server by calling serve on the same hostname and ip. 
This will create a parlai.agents.rag.retrieve_api.SearchEngineRetriever and try to connect 
and send a query, and parse the answer.

```bash
python search_server.py test_server --host 0.0.0.0:8080
```

### Testing the parser:

```bash
python search_server.py test_parser www.some_url_of_your_choice.com/
```


"""
import http.server
import json
import urllib.parse

import bs4
import fire
import html2text
import googlesearch
import parlai.agents.rag.retrieve_api
import rich
import rich.markup
import requests


print = rich.print

#_DEFAULT_HOST = "0.0.0.0"
#_DEFAULT_PORT = 8080
_DEFAULT_HOST = "0.0.0.0"
_DEFAULT_PORT = 8000

def _parse_host(host):
    splitted = host.split(":")
    hostname = splitted[0]
    port = splitted[1] if len(splitted) > 1 else _DEFAULT_PORT
    return hostname, port


def _get_content(url):
    resp = requests.get(url)
    page = resp.content

    soup = bs4.BeautifulSoup(page, features="lxml")
    pre_rendered = soup.find("title")
    title = (
        pre_rendered.renderContents().decode() if pre_rendered else "No Title"
    )

    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True
    text_maker.ignore_tables = True
    text_maker.ignore_images = True
    text_maker.ignore_emphasis = True
    text_maker.single_line = True
    text = text_maker.handle(page.decode("utf-8", errors="ignore"))
    output_dict = dict(url=url, title=title, content=text)
    print(
        f"title: `{rich.markup.escape(output_dict['title'])}`",
        f"url: {rich.markup.escape(output_dict['url'])}",
    )
    return output_dict


class SearchABC(http.server.BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers["Content-Length"])
        post_data = self.rfile.read(content_length)

        # query = urllib.parse.urlparse(post_data).query
        parsed = urllib.parse.parse_qs(post_data)
        for k, v in parsed.items():
            assert len(v) == 1, len(v)

        parsed = {k.decode(): v[0].decode() for k, v in parsed.items()}

        print(f"parsed: {parsed}")
        n = int(parsed["n"])
        q = parsed["q"]

        urls = self.search(q=q, n=n)

        output = json.dumps(
            dict(response=[_get_content(url) for url in urls])
        ).encode()
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.send_header("Content-Length", len(output))
        self.end_headers()
        self.wfile.write(output)

    def search(self, q, n):
        return NotImplemented(
            "Search is an abstract base class, not meant to be directly instantiated. "
            "You should instantiate a derived class like GoogleSearch."
        )
    

class GoogleSearchServer(SearchABC):
    def search(self, q, n):
        return googlesearch.search(q, num=n, stop=n)
    

class Application:
    def serve(self, host=_DEFAULT_HOST):
        host, port = _parse_host(host)

        with http.server.ThreadingHTTPServer((host, port), GoogleSearchServer) as server:
            print("Serving forever.")
            server.serve_forever()

    def test_parser(self, url):
        print(_get_content(url))

    def test_server(self, query, n, host=_DEFAULT_HOST):
        host, port = _parse_host(host)
        
        print(f"Query: `{query}`")
        print(f"n: {n}")

        retriever = parlai.agents.rag.retrieve_api.SearchEngineRetriever(
            dict(
                search_server=f"{host}:{port}",
                skip_retrieval_token=False,
            )
        )
        print("Retrieving one.")
        print(retriever._retrieve_single(query, n))
        print("Done.")


if __name__ == "__main__":
    fire.Fire(Application)

Overwriting search_server.py


In [ ]:
#!python search_server.py serve --host 0.0.0.0:8080
#!python search_server.py serve

In [ ]:
!python search_server.py test_parser https://www.google.co.jp

title: `Google` url: https://www.google.co.jp
{
    'url': 'https://www.google.co.jp',
    'title': 'Google',
    'content': 'Search Images Maps Play YouTube News Gmail Drive More »\n\nWeb 
History | Settings | Sign in\n\n  \n\n  \n  \n\n\n\n  \nAdvanced search  \n  \n 
\n\nGoogle offered in: {\n\nAdvertising ProgramsBusiness SolutionsAbout 
GoogleGoogle.com\n\n(C) 2021 - Privacy \\- Terms\n\n'
}


In [ ]:
get_ipython().system_raw(
    'python search_server.py serve &'
)

In [ ]:
!ps -ef | grep 8000 | grep -v grep

root          61      49  0 09:02 ?        00:00:13 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-fcc90645-5cd0-456a-b620-494046cd8880.json
root          81       1  0 09:02 ?        00:00:06 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 36925 --host 127.0.0.1 --port 22413 --server-access-token fbb463a7f05d2fe6f1896f56e2880523aa37c9a5eae817d56187013394fdce84
root         799      61  0 09:37 ?        00:00:00 /bin/bash -c ps -ef | grep 88
root         801     799  0 09:37 ?        00:00:00 grep 88


In [ ]:
#!curl http://0.0.0.0:8888
!curl http://0.0.0.0:8000

curl: (7) Failed to connect to 0.0.0.0 port 8000: Connection refused


In [ ]:
#!parlai interactive --model-file zoo:blenderbot2/blenderbot2_3B/model --search_server 0.0.0.0:8080
#!parlai interactive --model-file zoo:blenderbot2/blenderbot2_3B/model --search_server 0.0.0.0:8888

In [ ]:
!parlai interactive \
--model-file zoo:blenderbot2/blenderbot2_400M/model \
--search_server http://0.0.0.0:8000
#--model-file zoo:blenderbot2/blenderbot2_3B/model \
#--search-server http://asd

09:54:33 | Overriding opt["model_file"] to /content/ParlAI/data/models/blenderbot2/blenderbot2_400M/model (previously: /checkpoint/kshuster/projects/knowledge_bot/kbot_memfix_sweep25_Fri_Jul__9/338/model.oss)
09:54:33 | Overriding opt["search_server"] to http://0.0.0.0:8000 (previously: None)
09:54:33 | Using CUDA
09:54:33 | loading dictionary from /content/ParlAI/data/models/blenderbot2/blenderbot2_400M/model.dict
09:54:33 | num words = 50264
09:54:34 | BlenderBot2Fid: full interactive mode on.
09:55:13 | Creating the search engine retriever.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from 